In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from glob import glob
import pandas as pd
import pickle
from torch.utils.data import RandomSampler
import random
import scipy
import torch.nn.functional as F
from PIL import Image
from glob import glob
import wandb
import re
from adjustText import adjust_text
import seaborn as sns
import scipy
import statannot
import argparse
import sys
#print(glob('../../*'))x
sys.path.append('../../modified_medsam_repo')
from MedSAM_HCP.utils_hcp import *
from MedSAM_HCP.dataset import *
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
from MedSAM_HCP.MedSAM import convert_logits_to_preds_onehot

/gpfs/home/kn2347/.conda/envs/medsam/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [62]:
path = '/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_unet.csv'

df_hcp = pd.read_csv('/gpfs/home/kn2347/HCP_MedSAM_project/modified_medsam_repo/hcp_mapping_processed.csv')
df_desired = pd.read_csv('/gpfs/home/kn2347/HCP_MedSAM_project/modified_medsam_repo/darts_name_class_mapping_processed.csv')
label_converter = LabelConverter(df_hcp, df_desired)
#num_classes=1

train, val, test = load_datasets(
            path,
            '/gpfs/data/luilab/karthik/pediatric_seg_proj/train_val_test_split.pickle',
            label_id = None, bbox_shift=0, 
                sample_n_slices = None, label_converter=label_converter, NUM_CLASSES=103, 
                as_one_hot=True, pool_labels=False, dataset_type=MRIDataset_Imgs)

train_loader = DataLoader(train, batch_size=1, shuffle=False, num_workers=1)

In [127]:
weber_values = dict()
for label in range(0, 102):
    weber_values[label] = []
for i, (img, mask) in enumerate(tqdm(train_loader)):
    if i >= 256*50:
        break
    # calculate boundary pixels for mask
    #mask = mask[0]
    
    lol = torch.amax(mask, dim=(2,3)) == 1
    nei=None

    pixel_intensities = torch.mean(img, dim=1)

    for label in range(0, 102):
        if lol[0, label] == 1:
            # this label is present
            dilated = scipy.ndimage.binary_dilation(mask[0, label].numpy(), iterations=3).astype(np.float32)
            nei = dilated - mask[0, label].numpy()
            
            # get average value at the locations in pixel_intensities specified by nei
            nei = torch.tensor(nei).unsqueeze(0).unsqueeze(0)
            interested_pixels = nei * pixel_intensities
            interested_pixels_mean_intensity = torch.mean(interested_pixels[interested_pixels != 0])
            

            # get average value at locations specified by mask
            mask_pixel_intensities = mask[0, label] * pixel_intensities
            mask_pixel_intensities_mean_intensity = torch.mean(mask_pixel_intensities[mask_pixel_intensities != 0])

            weber_contrast = abs(mask_pixel_intensities_mean_intensity - interested_pixels_mean_intensity) / interested_pixels_mean_intensity
            weber_values[label].append(weber_contrast.item())

            
        
    


  6%|▌         | 12800/228096 [46:43<13:05:51,  4.57it/s]


In [1]:
# calculate weber_values[1] mean

medians = [np.median(weber_values[x]) for x in range(102)]
ci5 = [np.quantile(weber_values[x], 0.05) for x in range(102)]
ci95 = [np.quantile(weber_values[x], 0.95) for x in range(102)]

df_res = pd.DataFrame({'label': list(range(1,103)), 'median': medians, 'ci5': ci5, 'ci95': ci95})
#df_res['label'] = df_res['label'].apply(lambda x: label_converter.compressed_to_name(x))
#df_res = df_res.sort_values(by='median', ascending=False)
df_res.to_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/class_contrasts_12-30-24/using_50_mris.csv', index=False)

NameError: name 'np' is not defined

In [133]:
df_res

,label,median,ci5,ci95
0,1,0.305275,0.233013,0.362900
1,2,0.566297,0.380451,0.655262
2,3,0.414001,0.267896,0.534963
3,4,0.196934,0.114628,0.291474
4,5,0.253805,0.053328,0.953601
...,...,...,...,...
97,98,0.074961,0.006244,0.483958
98,99,0.068937,0.006073,0.224300
99,100,0.059677,0.005386,0.579900
100,101,0.048715,0.004485,0.174680


In [118]:
pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/training_set_class_statistics_10-15-24/class_statistics.csv')


,label,total_slices,total_pixels,avg_pixels_per_slice_when_present,avg_pixels_per_all_slices,fraction_slices_present
0,1,97036.0,196034640.0,2020.225896,859.439184,0.425417
1,2,26060.0,5570108.0,213.741673,24.420016,0.114250
2,3,12236.0,177081.0,14.472131,0.776344,0.053644
3,4,38779.0,12517295.0,322.785399,54.877310,0.170012
4,5,54601.0,50639640.0,927.448948,222.010206,0.239377
...,...,...,...,...,...,...
97,98,50700.0,12439773.0,245.360414,54.537445,0.222275
98,99,48432.0,11177503.0,230.787558,49.003503,0.212332
99,100,37867.0,9831934.0,259.643859,43.104368,0.166013
100,101,14903.0,948821.0,63.666443,4.159744,0.065337


In [120]:
pd.read_csv('/gpfs/home/kn2347/HCP_MedSAM_project/modified_medsam_repo/label_info.csv')

,label_number,name_without_side,side,group
0,Left-Cerebral-White-Matter,Cerebral-White-Matter,Left,White-Matter
1,Right-Cerebral-White-Matter,Cerebral-White-Matter,Right,White-Matter
2,Left-Cerebellum-White-Matter,Cerebellum-White-Matter,Left,White-Matter
3,Right-Cerebellum-White-Matter,Cerebellum-White-Matter,Right,White-Matter
4,Optic-Chiasm,Optic-Chiasm,Neither,White-Matter
...,...,...,...,...
97,CC_Anterior,CC_Anterior,Neither,Corpus-Callosum
98,CC_Central,CC_Central,Neither,Corpus-Callosum
99,CC_Mid_Anterior,CC_Mid_Anterior,Neither,Corpus-Callosum
100,CC_Mid_Posterior,CC_Mid_Posterior,Neither,Corpus-Callosum
